In [ ]:
# | default_exp _components.docs_dependencies

# Install docs dependencies

In [ ]:
# | export

import asyncio
from tempfile import TemporaryDirectory
import shutil
import sys

from fastkafka._components.logger import get_logger

In [ ]:
import os
from contextlib import contextmanager

import pytest

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
# | export


async def _check_npx(required_major_version: int = 9) -> None:
    if shutil.which("npx") is not None:
        cmd = "npx --version"
        proc = await asyncio.create_subprocess_shell(
            cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
        )
        output, _ = await proc.communicate()
        major_version = int(output.decode("UTF-8").split(".")[0])
        if major_version < required_major_version:
            raise RuntimeError(
                f"Found installed npx major version: {major_version}, required npx major version: {required_major_version}. To use documentation features of FastKafka, please update npx"
            ) 
    else:
        raise RuntimeError(
            f"npx not found, to use documentation generation features of FastKafka, you must have npx >= {required_major_version} installed"
        )

In [ ]:
@contextmanager
def _clean_path():
    path = os.environ["PATH"]
    try:
        os.environ["PATH"] = ""
        yield
    finally:
        os.environ["PATH"] = path

In [ ]:
await _check_npx()

In [ ]:
with _clean_path():
    with pytest.raises(RuntimeError) as e:
        await _check_npx()
        
assert e.value.args[0] == "npx not found, to use documentation generation features of FastKafka, you must have npx >= 9 installed"

In [ ]:
with pytest.raises(RuntimeError) as e:
    await _check_npx(required_major_version=999)
        
assert e.value.args[0] == "Found installed npx major version: 9, required npx major version: 999. To use documentation features of FastKafka, please update npx", e.value.args[0]

In [ ]:
# | export


async def _install_docs_deps() -> None:
    with TemporaryDirectory() as d:
        cmd = (
            "npx -y -p @asyncapi/generator ag https://raw.githubusercontent.com/asyncapi/asyncapi/master/examples/simple.yml @asyncapi/html-template -o "
            + d
        )

        proc = await asyncio.create_subprocess_shell(
            cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
        )
        stdout, stderr = await proc.communicate()

        if proc.returncode == 0:
            logger.info("AsyncAPI generator installed")
        else:
            logger.error("AsyncAPI generator NOT installed!")
            logger.info(
                f"stdout of '$ {cmd}'{stdout.decode('UTF-8')} \n return_code={proc.returncode}"
            )
            logger.info(
                f"stderr of '$ {cmd}'{stderr.decode('UTF-8')} \n return_code={proc.returncode}"
            )
            raise ValueError(
                f"""AsyncAPI generator NOT installed, used '$ {cmd}'
----------------------------------------
stdout:
{stdout.decode("UTF-8")}
----------------------------------------
stderr:
{stderr.decode("UTF-8")}
----------------------------------------
return_code={proc.returncode}"""
            )

In [ ]:
await _install_docs_deps()

[ERROR] __main__: AsyncAPI generator NOT installed!
[INFO] __main__: stdout of '$ npx -y -p @asyncapi/generator ag https://raw.githubusercontent.com/asyncapi/asyncapi/master/examples/simple.yml @asyncapi/html-template -o /tmp/tmp0bdul8m6'

Done! ✨
Check out your shiny new generated files at /tmp/tmp0bdul8m6.

 
 return_code=0
[INFO] __main__: stderr of '$ npx -y -p @asyncapi/generator ag https://raw.githubusercontent.com/asyncapi/asyncapi/master/examples/simple.yml @asyncapi/html-template -o /tmp/tmp0bdul8m6' 
 return_code=0


ValueError: AsyncAPI generator NOT installed, used '$ npx -y -p @asyncapi/generator ag https://raw.githubusercontent.com/asyncapi/asyncapi/master/examples/simple.yml @asyncapi/html-template -o /tmp/tmp0bdul8m6'
--------------------
stdout:
[32m

Done! ✨[0m
[33mCheck out your shiny new generated files at [0m[35m/tmp/tmp0bdul8m6[0m[33m.[0m


--------------------
stderr:

--------------------
return_code=0